In [ ]:
using Turing
using StatsPlots
using Optim

In [ ]:
include("read_data.jl")
include("plot_utils.jl")

# Iterative Bayesian Modeling with Probabilistic Programming

### Dataset: World-Wide Monthly Air Passengers

Data source: https://www.transtats.bts.gov/Data_Elements.aspx

In [ ]:
function get_map(res)
    m = res[argmax(vec(res[:lp]))]
    m.name_map.parameters, reshape(m.value.data, :)[1:length(m.name_map.parameters)] # discard internal parameters
end

In [ ]:
function base_plot(data)
    year_min = Int(floor(minimum(data[!,"Date"])))
    year_max = Int(ceil(maximum(data[!,"Date"])))
    p = plot(data[!,"Date"], data[!,"Total"],
        legend=false, xlabel="year", ylabel="passenger [10^7]", xticks=year_min:year_max, xrotation=45)
    return p
end

In [ ]:
base_plot(air_passengers_2013_2018)

### First Start: A Linear Trend Model

In [ ]:
@model function trend_model_1(x::Vector{Float64}, y::Vector{Float64})
    slope ~ Normal(0,3)
    intercept ~ Normal(6,3)
    error ~ InverseGamma(1,1)
    
    for i in eachindex(y)
        y[i] ~ Normal(slope * (x[i] - x[1]) + intercept, error)
    end
end

In [ ]:
upwards_trend_data = air_passengers_2013_2018[2009 .<= air_passengers_2013_2018[!,"Year"],:]
base_plot(upwards_trend_data)

In [ ]:
Turing.Random.seed!(0)
res = sample(trend_model_1(upwards_trend_data[!,"Date"], upwards_trend_data[!,"Total"]), NUTS(), 3000, progress=false)
res

In [ ]:
function plot_trend_model_1(data, slope, intercept, error)
    p = base_plot(data)
    t_min = data[!,"Date"][1]
    plot!(p, t -> slope * (t-t_min) + intercept, color="red")
    plot!(p, t -> slope * (t-t_min) + intercept + sqrt(error), color="orange")
    plot!(p, t -> slope * (t-t_min) + intercept - sqrt(error), color="orange")
    p
end
plot_trend_model_1(upwards_trend_data, mean(res).nt.mean...)

### Changing Trend

In [ ]:
function trend_model_2_f(t::Real, slope::Real, intercept::Real, changepoint::Real, adjustment::Real)
    k = slope
    m = intercept
    if changepoint ≤ t
        k += adjustment
        m -= changepoint * adjustment
    end
    return k * t + m
end

@model function trend_model_2(x::Vector{Float64}, y::Vector{Float64})
    slope ~ Normal(0,3)
    intercept ~ Normal(6,3)
    error ~ InverseGamma(1,1)
    changepoint ~ Uniform(0, x[end] - x[1])
    adjustment ~ Normal(0,1)
    for i in eachindex(y)
        y[i] ~ Normal(trend_model_2_f(x[i]-x[1], slope, intercept, changepoint, adjustment), error)
    end
end

In [ ]:
Turing.Random.seed!(0)
res = sample(trend_model_2(upwards_trend_data[!,"Date"], upwards_trend_data[!,"Total"]), NUTS(), 3000, progress=false)

In [ ]:
mean(res)
plot(res)

In [ ]:
function plot_trend_model_2(data, slope, intercept, error, changepoint, adjustment)
    p = base_plot(data)
    t_min = data[!,"Date"][1]
    plot!(p, t -> trend_model_2_f(t - t_min, slope, intercept, changepoint, adjustment), color="red")
    plot!(p, t -> trend_model_2_f(t - t_min, slope, intercept, changepoint, adjustment) + sqrt(error), color="orange")
    plot!(p, t -> trend_model_2_f(t - t_min, slope, intercept, changepoint, adjustment) - sqrt(error), color="orange")
    vline!([changepoint + t_min], linestyle=:dash, color="black")
    p
end
_, map_vector = get_map(res)
plot_trend_model_2(upwards_trend_data, map_vector...)

In [ ]:
@model function trend_model_3(x::Vector{Float64}, y::Vector{Float64})
    slope ~ Normal(0,3)
    intercept ~ Normal(6,3)
    error ~ InverseGamma(1,1)
    tau ~ InverseGamma(1,1)
    n_changepoints = length(y) ÷ 12 # changepoint at each year
    adjustments ~ filldist(Laplace(0,0.1), n_changepoints)
    
    k = slope
    m = intercept
    j = 1
    for i in eachindex(y)
        if i % 12 == 1
            k += adjustments[j]
            m -= (x[i] - x[1]) * adjustments[j]
            j += 1
        end
        y[i] ~ Normal(k * (x[i] - x[1]) + m, error + 1e-5)
    end
end

In [ ]:
Turing.Random.seed!(0)
res = sample(trend_model_3(air_passengers_2013_2018[!,"Date"], air_passengers_2013_2018[!,"Total"]), NUTS(), 3000, progress=false);

In [ ]:
function trend_model_3_f(t, slope, intercept, changepoints, adjustments)
    ix = changepoints .<= t
    return (slope + sum(adjustments[ix])) * t + (intercept - changepoints[ix]'adjustments[ix])
end

function plot_trend_model_3(data, slope, intercept, error, adjustments)
    p = base_plot(data)
    x = data[!,"Date"]
    t_min = x[1]
    changepoints = x[(1:length(x)) .% 12 .== 1] .- x[1]
    plot!(p, t -> trend_model_3_f(t - t_min, slope, intercept, changepoints, adjustments), color="red")
    plot!(p, t -> trend_model_3_f(t - t_min, slope, intercept, changepoints, adjustments) + sqrt(error), color="orange")
    plot!(p, t -> trend_model_3_f(t - t_min, slope, intercept, changepoints, adjustments) - sqrt(error), color="orange")

    vline!(changepoints[abs.(adjustments) .> 0.01] .+ t_min, linestyle=:dash, color="black")
    p
end
_, map_vector = get_map(res)
plot_trend_model_3(air_passengers_2013_2018, map_vector[1], map_vector[2], map_vector[3], map_vector[5:end])

In [ ]:
res = optimize(trend_model_3(air_passengers_2013_2018[!,"Date"], air_passengers_2013_2018[!,"Total"]), MAP(), Optim.Options(iterations=10_000))

In [ ]:
map_vector = res.values.array
plot_trend_model_3(air_passengers_2013_2018, map_vector[1], map_vector[2], map_vector[3], map_vector[5:end])

### Seasonality

In [ ]:
x = air_passengers_2013_2018[!,"Date"]
y = air_passengers_2013_2018[!,"Total"]
t_min = x[1]
changepoints = x[(1:length(x)) .% 12 .== 1] .- x[1]
slope, intercept, error, adjustments = map_vector[1], map_vector[2], map_vector[3], map_vector[5:end]

y_stationary =  y .- map(t ->  trend_model_3_f(t - t_min, slope, intercept, changepoints, adjustments), x)

mask = 1:48
plot(x[mask], y_stationary[mask], label="air passengers without trend")
plot!(t -> sin(2*pi*t), label="sin(t)")
plot!(t -> cos(2*pi*t), label="cos(t)")

In [ ]:
@model function seasonality_model(x::Vector{Float64}, y::Vector{Float64}, N_frequencies::Int)
    beta ~ filldist(Normal(0,1.), 2*N_frequencies)
    error ~ InverseGamma(1,1)
    
    for i in eachindex(y)
        t = x[i] - x[1]
        s = 0
        for n in 1:N_frequencies
            s += beta[2*n-1] * sin(2*pi*n*t)
            s += beta[2*n] * cos(2*pi*n*t)
        end
        y[i] ~ Normal(s, error)
    end

end

function seasonality_component(t::Float64, N_frequencies::Int, beta::Vector{<:Real})
    s = 0
    for n in 1:N_frequencies
        s += beta[2*n-1] * sin(2*pi*n*t)
        s += beta[2*n] * cos(2*pi*n*t)
    end
    return s
end

In [ ]:
N_frequencies = 3
res = optimize(seasonality_model(x, y_stationary, N_frequencies), MAP())
beta = res.values.array

mask = 1:60
plot(x[mask], y_stationary[mask], label="air passengers")
plot!(t -> seasonality_component(t, N_frequencies, beta), label="seasonality model MAP")


#### Combining the Trend and Seasonality Components

In [ ]:
@model function prophet(x::Vector{Float64}, y::Vector{Float64}, N_frequencies::Int)
    slope ~ Normal(0,3)
    intercept ~ Normal(6,3)
    error ~ InverseGamma(1,1)
    n_changepoints = Int(ceil(length(y) / 12)) # changepoint at each year
    tau ~ InverseGamma(1,1)
    adjustments ~ filldist(Laplace(0,tau+1e-5), n_changepoints)
    beta ~ filldist(Normal(0,1.), 2*N_frequencies)

    k = slope
    m = intercept
    j = 1
    for i in eachindex(y)
        t = (x[i] - x[1])
        if i % 12 == 1
            k += adjustments[j]
            m -= t * adjustments[j]
            j += 1
        end
        s = seasonality_component(t, N_frequencies, beta)

        y[i] ~ Normal(k * t + m + s, error+1e-5)
    end
end

In [ ]:
N_frequencies = 3
x = air_passengers_2013_2018[!,"Date"]
y = air_passengers_2013_2018[!,"Total"]

Turing.Random.seed!(0)
map_estimate = optimize(prophet(x, y, N_frequencies), MAP(), Optim.Options(iterations=10000))

In [ ]:
function prophet_model_f(t, slope, intercept, changepoints, adjustments, N_frequencies, beta)
    return trend_model_3_f(t, slope, intercept, changepoints, adjustments) + seasonality_component(t, N_frequencies, beta)
end

function plot_prophet_model(data, slope, intercept, error, adjustments, N_frequencies, beta)
    p = base_plot(data)
    x = data[!,"Date"]
    t_min = x[1]
    changepoints = x[(1:length(x)) .% 12 .== 1] .- x[1]
    plot!(p, t -> prophet_model_f(t - t_min, slope, intercept, changepoints, adjustments, N_frequencies, beta), color="red")
    # plot!(p, t -> prophet_model_f(t - t_min, slope, intercept, changepoints, adjustments, N_frequencies, beta) + sqrt(error), color="orange")
    # plot!(p, t -> prophet_model_f(t - t_min, slope, intercept, changepoints, adjustments, N_frequencies, beta) - sqrt(error), color="orange")

    vline!(changepoints[abs.(adjustments) .> 0.01] .+ t_min, linestyle=:dash, color="black")
    p
end

In [ ]:
map_vector = map_estimate.values.array
n_changepoints = length(y) ÷ 12

slope, intercept, error, tau,  adjustments = map_vector[1:4]..., map_vector[5:(5+n_changepoints-1)]
beta = map_vector[5+n_changepoints : end]

plot_prophet_model(air_passengers_2013_2018, slope, intercept, error, adjustments, N_frequencies, beta)

### Forecast

In [ ]:
mean(abs, adjustments), std(adjustments), tau

In [ ]:
mean(abs, rand(Laplace(0,tau),10^6))

In [ ]:
p_changepoint = mean(abs.(adjustments) .> 1e-3)
tau_changepoint = mean(abs,adjustments[abs.(adjustments) .> 1e-3])
p_changepoint, tau_changepoint

In [ ]:
mean(abs, rand(Bernoulli(p_changepoint),10^6) .* rand(Laplace(0,tau_changepoint),10^6))

In [ ]:
function prophet_forecast(x::Vector{Float64}, y::Vector{Float64},
    m::Float64, k::Float64, error::Float64, N_frequencies::Int, beta::Vector{Float64},
    forecast::Int)

    tau = mean(abs, adjustments)

    n_future_changepoints = ((length(y) + forecast) ÷ 12) - (length(y) ÷ 12)
    future_adjustments = rand(filldist(Laplace(0,tau), n_future_changepoints))

    y_pred = zeros(forecast)
    Δ = x[2] - x[1]
    x_future = x[end]
    i = length(y)+1
    j = 1
    while i <= length(y) + forecast
        x_future += Δ
        t = (x_future - x[1])

        if i % 12 == 1
            k += future_adjustments[j]
            m -= t * future_adjustments[j]
            j += 1
        end

        s = seasonality_component(t, N_frequencies, beta)
        y_pred[i-length(y)] = rand(Normal(k * t + m + s, error))
        i += 1
    end

    return y_pred
end

In [ ]:
Turing.Random.seed!(0)
res = sample(prophet(x, y, N_frequencies), NUTS(), 1000, progress=false, init_params=map_estimate.values.array)

In [ ]:
function sample_forecast(res, changepoints, forecast, n_samples_per_trace)
    n_changepoints = length(changepoints)
    y_pred = zeros(n_samples_per_trace * length(res), forecast)

    j = 0
    for i in 1:length(res)
        trace = res[i].value.data

        slope, intercept, error, tau,  adjustments = trace[1:4]..., trace[5:(5+n_changepoints-1)]
        beta = trace[5+n_changepoints : end]

        for _ in 1:n_samples_per_trace
            j += 1
            k = slope + sum(adjustments)
            m = intercept - adjustments'changepoints
            y_pred[j,:] = prophet_forecast(x,y,m,k,error,N_frequencies,beta,forecast)
        end
    end
    return y_pred
end

In [ ]:
function plot_forecast(x, y, x_forecast, y_pred)
    forecast = length(x_forecast)
    p = plot(x, y, label="air passenger historic data", xlabel="year", ylabel="passenger [10^7]", legend=:topleft)
    q05 = map(i -> quantile(y_pred[:,i], 0.05), 1:forecast)
    q50 = map(i -> quantile(y_pred[:,i], 0.5), 1:forecast)
    q95 = map(i -> quantile(y_pred[:,i], 0.95), 1:forecast)
    plot!(x_forecast, q50, ribbon=(q50-q05,q95-q50), label="forecast")
    return p
end

In [ ]:
n_year_forecast = 4
forecast = n_year_forecast * 12
x_forecast = collect(maximum(x) .+ (1:forecast)./12)

y_pred = sample_forecast(res, forecast, 10)
plot_forecast(x, y, x_forecast, y_pred)

### Covid ....

In [ ]:
base_plot(air_passengers_2013_2023)

In [ ]:
N_frequencies = 3
x = air_passengers_2013_2023[!,"Date"]
y = air_passengers_2013_2023[!,"Total"]
changepoints = x[(1:length(x)) .% 12 .== 1] .- x[1]

Turing.Random.seed!(0)
map_estimate = optimize(prophet(x, y, N_frequencies), MAP(), Optim.Options(iterations=10000))

Turing.Random.seed!(0)
res = sample(prophet(x, y, N_frequencies), NUTS(), 1000, progress=false, init_params=map_estimate.values.array)

In [ ]:
map_vector = map_estimate.values.array
n_changepoints = length(y) ÷ 12

slope, intercept, error, tau,  adjustments = map_vector[1:4]..., map_vector[5:(5+n_changepoints-1)]
beta = map_vector[5+n_changepoints : end]

plot_prophet_model(air_passengers_2013_2023, slope, intercept, error, adjustments, N_frequencies, beta)

In [ ]:
n_year_forecast = 4
forecast = n_year_forecast * 12
x_forecast = collect(maximum(x) .+ (1:forecast)./12)

y_pred = sample_forecast(res, changepoints, forecast, 10)
plot_forecast(x, y, x_forecast, y_pred)

In [ ]:
@model function prophet_covid(x::Vector{Float64}, y::Vector{Float64}, N_frequencies::Int)
    slope ~ Normal(0,3)
    intercept ~ Normal(6,3)
    error ~ InverseGamma(1,1)
    n_changepoints = Int(ceil(length(y) / 12)) # changepoint at each year
    tau ~ InverseGamma(1,1)
    adjustments ~ filldist(Laplace(0,tau+1e-5), n_changepoints)
    beta ~ filldist(Normal(0,1.), 2*N_frequencies)

    shock_2020 ~ Normal(-8,1)
    shock_2021 ~ Normal(-4,1)
    shock_2022 ~ Normal(-2,1)

    k = slope
    m = intercept
    j = 1
    for i in eachindex(y)
        t = (x[i] - x[1])
        if i % 12 == 1
            k += adjustments[j]
            m -= t * adjustments[j]
            j += 1
        end
        s = seasonality_component(t, N_frequencies, beta)


        if 2020 ≤ x[i] && x[i] < 2021
            d = shock_2020
        elseif 2021 ≤ x[i] && x[i] < 2022
            d = shock_2021
        elseif 2022 ≤ x[i] && x[i] < 2023
            d = shock_2022
        else
            d = 0
        end

        y[i] ~ Normal(k * t + m + s + d, error+1e-5)
    end
end

In [ ]:
N_frequencies = 3
x = air_passengers_2013_2023[!,"Date"]
y = air_passengers_2013_2023[!,"Total"]
changepoints = x[(1:length(x)) .% 12 .== 1] .- x[1]

Turing.Random.seed!(0)
map_estimate = optimize(prophet_covid(x, y, N_frequencies), MAP(), Optim.Options(iterations=10000))

Turing.Random.seed!(0)
res = sample(prophet_covid(x, y, N_frequencies), NUTS(), 1000, progress=false, init_params=map_estimate.values.array)

In [ ]:
function shock_component(t::Float64, shocks::Vector{Float64})
    if 2020 ≤ t && t < 2021
        d = shocks[1]
    elseif 2021 ≤ t && t < 2022
        d = shocks[2]
    elseif 2022 ≤ t && t < 2023
        d = shocks[3]
    else
        d = 0
    end
end

function plot_prophet_covid_model(data, slope, intercept, error, adjustments, N_frequencies, beta, shocks)
    p = base_plot(data)
    x = data[!,"Date"]
    t_min = x[1]
    changepoints = x[(1:length(x)) .% 12 .== 1] .- x[1]
    plot!(p, t -> prophet_model_f(t - t_min, slope, intercept, changepoints, adjustments, N_frequencies, beta) + shock_component(t,shocks), color="red")

    vline!(changepoints[abs.(adjustments) .> 0.01] .+ t_min, linestyle=:dash, color="black")
    p
end

In [ ]:
map_vector = map_estimate.values.array
n_changepoints = length(y) ÷ 12

slope, intercept, error, tau,  adjustments = map_vector[1:4]..., map_vector[5:(5+n_changepoints-1)]
beta = map_vector[5+n_changepoints : 5+n_changepoints + 2*N_frequencies-1]
shocks = map_vector[5+n_changepoints + 2*N_frequencies : end]

plot_prophet_covid_model(air_passengers_2013_2023, slope, intercept, error, adjustments, N_frequencies, beta, shocks)

In [ ]:
n_year_forecast = 4
forecast = n_year_forecast * 12
x_forecast = collect(maximum(x) .+ (1:forecast)./12)

y_pred = sample_forecast(res, changepoints, forecast, 10)
plot_forecast(x, y, x_forecast, y_pred)